## Overview

To facilitate graph analysis, we need to support sharing layouts between graphs, something I haven't seen done very well elsewhere.  In particular, it should be possible to:

* Capture the layout from a graph $G_0$, either:
    * &#x2713; After creating the graph mark.
    * Without creating a graph mark.
* Use that layout to render a graph $G_1$.  This should work in all cases:
    * &#x2713; $G_1$ has the same vertices as $G_0$, in the same order.
    * $G_1$ has the same vertices as $G_0$, in a different order.
    * $G_1$ has a subset of the vertices in $G_0$, in arbitrary order.
    * $G_1$ has a superset of the vertices in $G_0$, in arbitrary order.
        * In this case the new vertices should be laid-out using the chosen layout algorithm, without disturbing the existing vertices.
* An implicit requirement is the ability to perform a layout on some vertices while leaving others unchanged.

To create a layout without creating a graph mark:

```python
layout = toyplot.layout.graph(edges) 
layout = toyplot.layout.graph(edges, vcount) 
layout = toyplot.layout.graph(edges, vcoordinates) 
layout = toyplot.layout.graph(sources, targets) 
layout = toyplot.layout.graph(sources, targets, vcount) 
layout = toyplot.layout.graph(sources, targets, vcoordinates) 
```

The returned layout object has a subset of the properties of toyplot.mark.Graph:

```python
layout.vcount
layout.vid
layout.vcoordinates
layout.ecount
layout.esource
layout.etarget
layout.eshape
layout.ecoordinates
layout.edges?
```

In [1]:
import numpy
import toyplot.color
import toyplot.generate

In [2]:
numpy.random.seed(1234)

In [3]:
# Random graph
#vertices = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
#graph = numpy.random.choice(vertices, (40, 2))

graph = toyplot.generate.prufer_tree(numpy.random.choice(4, 12))

# Reingold tree
#graph = numpy.array([[0,1],[0,2],[1,3],[2,4],[2,5],[3,6],[4,7],[5,8],[5,9],[6,10],[7,11],[9,12],[10,13],[12,14]])

In [4]:
%%time

colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "red"]))

canvas = toyplot.Canvas(width=1000, height=500)
axes = canvas.axes(grid=(1, 2, 0), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    graph,
    #layout=toyplot.layout.Random(),
    #layout=toyplot.layout.Eades(),
    #layout=toyplot.layout.FruchtermanReingold(M=100),
    layout=toyplot.layout.Buchheim(),
    #layout=toyplot.layout.GraphViz(),
    vcolor=colormap,
    vmarker="o",
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

vcoordinates = mark.vcoordinates.copy()
axes.text(vcoordinates.T[0], vcoordinates.T[1], mark.vid, color="black")

vcoordinates[4:,:] = numpy.ma.masked

axes = canvas.axes(grid=(1, 2, 1), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    graph,
    vcoordinates,
    #layout=toyplot.layout.Random(seed=125),
    #layout=toyplot.layout.Eades(),
    #layout=toyplot.layout.FruchtermanReingold(M=100),
    layout=toyplot.layout.Buchheim(),
    #layout=toyplot.layout.GraphViz(),
    vcolor=colormap,
    vmarker="o",
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
vcoordinates = mark.vcoordinates.copy()
axes.text(vcoordinates.T[0], vcoordinates.T[1], mark.vid, color="black")

INFO:toyplot:Graph layout time: 0.60510635376 ms
INFO:toyplot:Graph layout time: 0.351905822754 ms


CPU times: user 17 ms, sys: 2.73 ms, total: 19.7 ms
Wall time: 18.2 ms
